In [1]:
import cv2
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import os

IMG_SAVE_PATH = 'image_data'

CLASS_MAP = {
    "rock": 0,
    "paper": 1,
    "scissors": 2,
    "none": 3
}

NUM_CLASSES = len(CLASS_MAP)


def mapper(val):
    return CLASS_MAP[val]


def get_model():
    model = Sequential([
        SqueezeNet(input_shape=(227, 227, 3), include_top=False),
        Dropout(0.5),
        Convolution2D(NUM_CLASSES, (1, 1), padding='valid'),
        Activation('relu'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])
    return model


# load images from the directory
dataset = []
for directory in os.listdir(IMG_SAVE_PATH):
    path = os.path.join(IMG_SAVE_PATH, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        # to make sure no hidden files get in our way
        if item.startswith("."):
            continue
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (227, 227))
        dataset.append([img, directory])

'''
dataset = [
    [[...], 'rock'],
    [[...], 'paper'],
    ...
]
'''
data, labels = zip(*dataset)
labels = list(map(mapper, labels))


'''
labels: rock,paper,paper,scissors,rock...
one hot encoded: [1,0,0], [0,1,0], [0,1,0], [0,0,1], [1,0,0]...
'''

# one hot encode the labels
labels = np_utils.to_categorical(labels)

# define the model
model = get_model()
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# start training
model.fit(np.array(data), np.array(labels), epochs=10)

# save the model for later use
model.save("rock-paper-scissors-model.h5")


Using TensorFlow backend.


Epoch 1/10
1000/1000 [==============================] - 41s 41ms/step - loss: 0.6978 - accuracy: 0.7790
Epoch 2/10
1000/1000 [==============================] - 43s 43ms/step - loss: 0.0205 - accuracy: 0.9920
Epoch 3/10
1000/1000 [==============================] - 44s 44ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 4/10
1000/1000 [==============================] - 47s 47ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 5/10
1000/1000 [==============================] - 47s 47ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 6/10
1000/1000 [==============================] - 48s 48ms/step - loss: 8.3137e-04 - accuracy: 1.0000
Epoch 7/10
1000/1000 [==============================] - 53s 53ms/step - loss: 2.7868e-04 - accuracy: 1.0000
Epoch 8/10
1000/1000 [==============================] - 49s 49ms/step - loss: 2.3705e-04 - accuracy: 1.0000
Epoch 9/10
1000/1000 [==============================] - 50s 50ms/step - loss: 2.6347e-04 - accuracy: 1.0000
Epoch 10/10
1000/1000 [=========================